For week 6 of 2019, Carl and I are going to get you playing with aggregation and, more importantly, introduce you all to our side hustle: Chin & Beard Suds Co – esteemed purveyor of (fake) luxury soap products.

# The Scenario:

Chin & Beard Suds Co (C&BS Co.) is looking to update their company data table which details the monthly profit generated by each product in Scotland and England. This has already been done for the first three months of 2019 for Scotland, but the March sales information for England has yet to be aggregated and added to the master table.

<img src="https://3.bp.blogspot.com/-hXgEMvkx6Sc/XJEY5H9ujuI/AAAAAAAAAJk/dS7kwKV61X87EdasxB2Uv7HCkYBIvhUggCLcBGAs/s1600/CompanyData.PNG" alt="Alternative text" />

The March sales information for England is in its own table and contains how many units of each product type were sold in each city.

<img src="https://4.bp.blogspot.com/-WuDGGEmZqPc/XJEY5AK8y-I/AAAAAAAAAJc/gNfWBNbtERIiuIF34oGg9Hvzfsza4pbSQCEwYBhgL/s1600/EnglandMarchSales.PNG" alt="Alternative text" />

The pricing and production costs for each product type are also contained in their own sheet.

<img src="https://3.bp.blogspot.com/-N7Myk8FCeI8/XJEY5L37YDI/AAAAAAAAAJg/gvy6d3zIcxQ7flFeqOnTxhUGG7HurMrsACEwYBhgL/s320/ProductDetails.PNG" alt="Alternative text" />

# The Requirements:

- Import the file.
- Make sure to use all 3 sheets.
- Use the soap pricing details to calculate the profit in each city in England in March.
- Add the missing overall profit for each product in England in March to the Company Data table.
- Note: The C&BS Co monthly sales period starts on the 19th of each month.


# The Output

- 4 Columns
- 12 Rows
- One row for each country’s monthly profit for each product type.

In [161]:
import pandas as pd
import numpy as np
from datetime import datetime

In [162]:
df1 = pd.read_excel('Week6Input.xlsx', sheet_name='Company Data')
print(df1)

       Month   Country     Category  Profit
0 2019-01-19  Scotland     Bar Soap  1475.0
1 2019-02-19  Scotland     Bar Soap  1350.0
2 2019-03-19  Scotland     Bar Soap  1550.0
3 2019-01-19  Scotland  Liquid Soap   755.0
4 2019-02-19  Scotland  Liquid soap   695.0
5 2019-03-19  Scotland  Liquid Soap   645.0
6 2019-01-19   England     Bar Soap  1955.0
7 2019-02-19   England     Bar Soap  2145.0
8 2019-01-19   England  Liquid Soap  1630.0
9 2019-02-19   England  Liquid Soap  1755.0


In [163]:
df2 = pd.read_excel('Week6Input.xlsx', sheet_name='England - Mar 2019')
print(df2)

   Country     Category        City  Units Sold
0  England     Bar Soap      London       475.0
1  England  Liquid Soap      London       364.0
2  England     Bar Soap  Manchester       195.0
3  England  Liquid Soap  Manchester       223.0
4  England     Bar Soap        York       275.0
5  England  Liquid Soap        York       327.0


In [164]:
df3 = pd.read_excel('Week6Input.xlsx', sheet_name='Soap Pricing Details')
print(df3)

  Type of Soap  Manufacturing Cost per Unit  Selling Price per Unit
0         Bar                           0.8                     3.0
1       Liquid                          0.2                     1.2


In [165]:
type_of_soap_array = []
for index, series in df2.iterrows():
    if 'Bar' in df2.loc[index,'Category']:
        type_of_soap_array.append('Bar')
    elif 'Liquid' in df2.loc[index,'Category']:
        type_of_soap_array.append('Liquid')
df2['Type of Soap'] = type_of_soap_array
print(df2)


   Country     Category        City  Units Sold Type of Soap
0  England     Bar Soap      London       475.0          Bar
1  England  Liquid Soap      London       364.0       Liquid
2  England     Bar Soap  Manchester       195.0          Bar
3  England  Liquid Soap  Manchester       223.0       Liquid
4  England     Bar Soap        York       275.0          Bar
5  England  Liquid Soap        York       327.0       Liquid


In [166]:
df3['Type of Soap'] = df3['Type of Soap'].str.strip()

In [167]:
df2_join_df3 = pd.merge(df2, df3, how='left', on='Type of Soap')
print(df2_join_df3)

   Country     Category        City  Units Sold Type of Soap  \
0  England     Bar Soap      London       475.0          Bar   
1  England  Liquid Soap      London       364.0       Liquid   
2  England     Bar Soap  Manchester       195.0          Bar   
3  England  Liquid Soap  Manchester       223.0       Liquid   
4  England     Bar Soap        York       275.0          Bar   
5  England  Liquid Soap        York       327.0       Liquid   

   Manufacturing Cost per Unit  Selling Price per Unit  
0                          0.8                     3.0  
1                          0.2                     1.2  
2                          0.8                     3.0  
3                          0.2                     1.2  
4                          0.8                     3.0  
5                          0.2                     1.2  


In [168]:
df2_join_df3['Profit'] = (df2_join_df3['Selling Price per Unit'] - df2_join_df3['Manufacturing Cost per Unit']) * df2_join_df3['Units Sold']
df2_join_df3.drop(columns=['Manufacturing Cost per Unit','Selling Price per Unit'], inplace=True)
print(df2_join_df3)

   Country     Category        City  Units Sold Type of Soap  Profit
0  England     Bar Soap      London       475.0          Bar  1045.0
1  England  Liquid Soap      London       364.0       Liquid   364.0
2  England     Bar Soap  Manchester       195.0          Bar   429.0
3  England  Liquid Soap  Manchester       223.0       Liquid   223.0
4  England     Bar Soap        York       275.0          Bar   605.0
5  England  Liquid Soap        York       327.0       Liquid   327.0


In [169]:
df4 = df2_join_df3.groupby(['Category', 'Country']).agg({'Profit': 'sum'})
df4.reset_index(inplace=True)
print(df4)

      Category  Country  Profit
0     Bar Soap  England  2079.0
1  Liquid Soap  England   914.0


In [170]:
df4['Month'] = datetime(2019,3,19)
print(df4)

      Category  Country  Profit      Month
0     Bar Soap  England  2079.0 2019-03-19
1  Liquid Soap  England   914.0 2019-03-19


In [171]:
result = pd.concat([df1,df4])
result.reset_index(inplace=True)
result.drop(columns=['index'], inplace=True)
print(result)

        Month   Country     Category  Profit
0  2019-01-19  Scotland     Bar Soap  1475.0
1  2019-02-19  Scotland     Bar Soap  1350.0
2  2019-03-19  Scotland     Bar Soap  1550.0
3  2019-01-19  Scotland  Liquid Soap   755.0
4  2019-02-19  Scotland  Liquid soap   695.0
5  2019-03-19  Scotland  Liquid Soap   645.0
6  2019-01-19   England     Bar Soap  1955.0
7  2019-02-19   England     Bar Soap  2145.0
8  2019-01-19   England  Liquid Soap  1630.0
9  2019-02-19   England  Liquid Soap  1755.0
10 2019-03-19   England     Bar Soap  2079.0
11 2019-03-19   England  Liquid Soap   914.0
